In [1]:
import sys
import os

# Get the absolute path to the `src` directory
src_path = os.path.abspath(os.path.join('..', 'src'))

# Add the `src` directory to the Python path
if src_path not in sys.path:
    sys.path.append(src_path)

from fetch_news import fetch_articles, extract_text

ticker = 'TSLA'

articles = fetch_articles(ticker, '2024-01-01', '2024-01-20')
texts = extract_text(articles)

import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import pandas as pd

tokenizer = AutoTokenizer.from_pretrained("yiyanghkust/finbert-tone")
model = AutoModelForSequenceClassification.from_pretrained("yiyanghkust/finbert-tone")

def analyze_sentiment(texts):
    sentiments = []
    for text in texts:
        inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
        outputs = model(**inputs)
        scores = torch.nn.functional.softmax(outputs.logits, dim=-1)

        # Compute a single score: -1 (negative) to 1 (positive)
        sentiment_score = scores[0][0].item() - scores[0][2].item()  # Positive - Negative

        sentiments.append({
            'text': text,
            'positive': scores[0][0].item(),
            'neutral': scores[0][1].item(),
            'negative': scores[0][2].item(),
            'sentiment_score': sentiment_score
        })
    return sentiments


sentiments = analyze_sentiment(texts)

df = pd.DataFrame(sentiments)

# Calculate the average sentiment score
average_sentiment_score = df['sentiment_score'].mean()

# Print the result
print(f"Average Sentiment Score for {ticker}: {average_sentiment_score:.2f}")


Average Sentiment Score for TSLA: -0.08
